In [1]:
import pandas as pd
import numpy as np
import requests

In [101]:
requests.get('https://www.imdb.com/title/tt0111161/', headers={'User-Agent': 'Mozilla/5.0'}).content

b'<!DOCTYPE html><html lang="en-US" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><script>if(typeof uet === \'function\'){ uet(\'bb\', \'LoadTitle\', {wb: 1}); }</script><script>window.addEventListener(\'load\', (event) => {\n        if (typeof window.csa !== \'undefined\' && typeof window.csa === \'function\') {\n            var csaLatencyPlugin = window.csa(\'Content\', {\n                element: {\n                    slotId: \'LoadTitle\',\n                    type: \'service-call\'\n                }\n            });\n            csaLatencyPlugin(\'mark\', \'clickToBodyBegin\', 1679150840039);\n        }\n    })</script><title>Um Sonho de Liberdade (1994) - IMDb</title><meta name="description" content="Um Sonho de Liberdade: Directed by Frank Darabont. With Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler. Over the course of several years, two con

# Web Scrapping

Muitas vezes os dados que queremos não estão disponibilizados através de APIs, apenas sites. Neste momento precisamos recorrer às ferramentas de **Web Scrapping**!

*Web scrapping* é a extração de informação estruturada a partir de paginas na internet: por exemplo, podemos extrair todos os artigos de um jornal que mencionem um certo produto, ou então as informações de uma série de tabelas da Wikipedia.

Hoje vamos aprender como utilizar as bibliotecas BeautifulSoup e Selenium para extrair informações a partir de links específicos, realizar buscas e navegar páginas.

# Conhecendo o BeautifulSoup

Vamos começar extraindo informações básicas a partir de uma notícia do portal UOL. O primeiro passo é utilizar a biblioteca `requests` para *baixar* o html da página:

In [2]:
url = "https://www.uol.com.br/esporte/futebol/ultimas-noticias/2022/01/22/em-1995-decisao-na-base-entre-palmeiras-x-sp-terminou-em-morte-no-pacaembu.htm"

In [4]:
response = requests.get(url)



In [5]:
response

<Response [200]>

In [6]:
html_str = response.text
type(html_str)

str

In [7]:
print(html_str[0:1000])


<!DOCTYPE html> <html lang="pt-br"> <head><meta charset="utf-8"><meta http-equiv="Content-Type" content="text/html; charset=utf-8">   <title>Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte</title><link rel="preconnect" href="https://stc.uol.com" crossorigin="anonymous"><link rel="preconnect" href="https://c.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.imguol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://me.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous"><link rel="dns-prefetch" href="https://stc.uol.com"><link rel="dns-prefetch" href="https://c.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.imguol.com.br"><link rel="dns-prefetch" href="h

In [8]:
html_bytes = response.content
type(html_bytes)


bytes

In [9]:
print(html_bytes[0:1000])


b'<!DOCTYPE html> <html lang="pt-br"> <head><meta charset="utf-8"><meta http-equiv="Content-Type" content="text/html; charset=utf-8">   <title>Em 1995, decis\xc3\xa3o na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte</title><link rel="preconnect" href="https://stc.uol.com" crossorigin="anonymous"><link rel="preconnect" href="https://c.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.imguol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://me.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous"><link rel="dns-prefetch" href="https://stc.uol.com"><link rel="dns-prefetch" href="https://c.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.imguol.com.br"><link rel="dns-prefetch

## Transfome em sopa

A página extraída acima é apenas um string: o código HTML. Para utiliza-la dentro do Python, precisamos de um **parser**: um conjunto de funções que nos permite **interpretar** este código HTML e extrair informações relevantes. A biblioteca *BeautifulSoup* implementa um **parser** de HTML dentro do Python, dando acesso à arvore de tags (`<head>`, `<link ...> `, etc).

Para utilizar este **parser** precisamos entender um pouco da estrutura de um arquivo HTML. Mas antes vamos utilizar o BeautifulSoup para deixar o *print* de nosso HTML mais organizado:

In [ ]:
!pip3 install bs4

In [10]:
from bs4 import BeautifulSoup


In [11]:
soup = BeautifulSoup(html_bytes)


In [12]:
type(soup)


bs4.BeautifulSoup

In [13]:
print(soup.prettify())


<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte
  </title>
  <link crossorigin="anonymous" href="https://stc.uol.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://c.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://conteudo.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://conteudo.imguol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://me.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="https://stc.uol.com" rel="dns-prefetch"/>
  <link href="https://c.jsuol.com.br" rel="dns-prefetch"/>
  <link href="https://conteudo.jsuol.com.br" rel="dns-prefetch"/>
  <link href="https://conteudo.imguol.com.

## Encontrando Tags

Um arquivo HTML é estruturado em **tags**: marcações com o formato `<nome_do_tag>` e `</nome_do_tag>`. A primeira denota o **inicio do conteúdo** do tag, a segunda o **fim do conteúdo**. Vamos entender como um **tag simples** funciona analisando o conteúdo do `<title>`.

Para encontrar todas as ocorrências de um **tag** pelo seu **nome** utilizamos o método `.find_all()`:

In [14]:
soup.find_all("title")


[<title>Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte</title>]

O método nos retornou todas as ocorrências de `<title>` no HTML (uma só no caso) em uma lista. Vamos ver o que essa lista contém:

In [15]:
todos_titles = soup.find_all("title")
print(type(todos_titles))

<class 'bs4.element.ResultSet'>


In [17]:
type(todos_titles[0])

bs4.element.Tag

In [18]:
title = soup.find_all("title")[0]
print(type(title))


<class 'bs4.element.Tag'>


O objeto `Tag` da BeautifulSoup contém todas as informações de um tag: atributos, links, conteúdo... Por enquanto vamos olhar o conteúdo desse tag (o que está entre `<title>` e `< \title>`) utilizando o atributo `.text`:

In [19]:
title.text


'Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte'

In [20]:
type(title.text)


str

### **Tags** com mais que uma ocorrência 

A maior parte dos **tags** ocorrem múltiplas vezes em um documento. Vamos buscar um **tag** com essa característica.

In [21]:
headers = soup.find_all("h3")
print(headers)


[<h3 class="thumb-title"> Advogado da vítima diz que Robinho busca 'pelo em ovo' para evitar cadeia  </h3>, <h3 class="thumb-title"> Invasão faz Corinthians reavaliar volta de Luan e demissão de dirigentes  </h3>, <h3 class="thumb-title"> São Paulo em crise, há clima para o Rogério Ceni? Com Mauro, Juca, Dan Stulbach, Arnaldo e Tironi  </h3>, <h3>Ocorreu um erro ao carregar os comentários.</h3>, <h3>Essa discussão está encerrada</h3>, <h3 class="h-components collection-title custom-title">Futebol</h3>, <h3 class="thumb-title title-xsmall title-lg-small">Ceni, Marcos Paulo e as relações humanas que insistem em entrar em campo</h3>, <h3 class="thumb-title title-xsmall title-lg-small">Cristiano Ronaldo vira motivo de polêmica no exército da Colômbia; entenda</h3>, <h3 class="thumb-title title-xsmall title-lg-small">América-MG x Cruzeiro: confira onde assistir à semifinal do Mineiro</h3>, <h3 class="thumb-title title-xsmall title-lg-small">O Atlético-MG não tem o direito de cair para o Ath

In [22]:
len(headers)

17

In [23]:
print(headers[0])


<h3 class="thumb-title"> Advogado da vítima diz que Robinho busca 'pelo em ovo' para evitar cadeia  </h3>


In [24]:
print(headers[0].text)


 Advogado da vítima diz que Robinho busca 'pelo em ovo' para evitar cadeia  


In [26]:
type(headers[0])

bs4.element.Tag

In [27]:
# EXERCICIO
# Utilize uma list comprehension para criar uma lista com o texto de todos os headers h3
[title.text for title in soup.find_all("h3")]

[" Advogado da vítima diz que Robinho busca 'pelo em ovo' para evitar cadeia  ",
 ' Invasão faz Corinthians reavaliar volta de Luan e demissão de dirigentes  ',
 ' São Paulo em crise, há clima para o Rogério Ceni? Com Mauro, Juca, Dan Stulbach, Arnaldo e Tironi  ',
 'Ocorreu um erro ao carregar os comentários.',
 'Essa discussão está encerrada',
 'Futebol',
 'Ceni, Marcos Paulo e as relações humanas que insistem em entrar em campo',
 'Cristiano Ronaldo vira motivo de polêmica no exército da Colômbia; entenda',
 'América-MG x Cruzeiro: confira onde assistir à semifinal do Mineiro',
 'O Atlético-MG não tem o direito de cair para o Athletic. E não vai!',
 'Dupla decisiva! Yuri Alberto e Róger Guedes dominam estatísticas ofensivas do Corinthians no Paulistão',
 'Santos tem apenas 11% de aproveitamento contra clubes da Série A na temporada',
 'Desfalcado, São Paulo não terá problemas com limite de estrangeiros nos torneios nacionais',
 'Com janela perto do fim, Corinthians corre para contra

### Buscando múltiplos **tags**

Além de buscar **tags** um a um, podemos utilizar o método `.find_all()` para encontrar todas as ocorrências de uma lista de tags:

In [28]:
lista_tags = ["h1", "h2", "h3"]
todos_headers = soup.find_all(lista_tags)
print(todos_headers)


[<h2 class="title-name"> <a data-audience-click='{"reference":"titulo-colecao","component":"title","mediaName":"Title"}' href="https://www.uol.com.br/esporte/futebol/ultimas/">Futebol</a> </h2>, <h1 class=""> <span> <i class="col-sm-22 col-md-22 col-lg-22 custom-title" ia-title="">Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu  </i></span> </h1>, <h3 class="thumb-title"> Advogado da vítima diz que Robinho busca 'pelo em ovo' para evitar cadeia  </h3>, <h3 class="thumb-title"> Invasão faz Corinthians reavaliar volta de Luan e demissão de dirigentes  </h3>, <h3 class="thumb-title"> São Paulo em crise, há clima para o Rogério Ceni? Com Mauro, Juca, Dan Stulbach, Arnaldo e Tironi  </h3>, <h3>Ocorreu um erro ao carregar os comentários.</h3>, <h3>Essa discussão está encerrada</h3>, <h3 class="h-components collection-title custom-title">Futebol</h3>, <h3 class="thumb-title title-xsmall title-lg-small">Ceni, Marcos Paulo e as relações humanas que insistem em entrar

In [31]:
todos_headers[0]

<h2 class="title-name"> <a data-audience-click='{"reference":"titulo-colecao","component":"title","mediaName":"Title"}' href="https://www.uol.com.br/esporte/futebol/ultimas/">Futebol</a> </h2>

Podemos utilizar o atributo `.name` para determinar qual o tipo de cada tag em nossa lista:

In [32]:
print(todos_headers[0].name)
print(todos_headers[0].text)


h2
 Futebol 


In [33]:
todos_headers[0]

<h2 class="title-name"> <a data-audience-click='{"reference":"titulo-colecao","component":"title","mediaName":"Title"}' href="https://www.uol.com.br/esporte/futebol/ultimas/">Futebol</a> </h2>

In [35]:
# EXERCICIO
# Utilize uma list comprehension para criar uma lista
# de uplas (tipo do tag, conteúdo)
[(header.name, header.text.strip()) for header in todos_headers]

[('h2', 'Futebol'),
 ('h1',
  'Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu'),
 ('h3',
  "Advogado da vítima diz que Robinho busca 'pelo em ovo' para evitar cadeia"),
 ('h3',
  'Invasão faz Corinthians reavaliar volta de Luan e demissão de dirigentes'),
 ('h3',
  'São Paulo em crise, há clima para o Rogério Ceni? Com Mauro, Juca, Dan Stulbach, Arnaldo e Tironi'),
 ('h3', 'Ocorreu um erro ao carregar os comentários.'),
 ('h3', 'Essa discussão está encerrada'),
 ('h3', 'Futebol'),
 ('h3',
  'Ceni, Marcos Paulo e as relações humanas que insistem em entrar em campo'),
 ('h3',
  'Cristiano Ronaldo vira motivo de polêmica no exército da Colômbia; entenda'),
 ('h3', 'América-MG x Cruzeiro: confira onde assistir à semifinal do Mineiro'),
 ('h3', 'O Atlético-MG não tem o direito de cair para o Athletic. E não vai!'),
 ('h3',
  'Dupla decisiva! Yuri Alberto e Róger Guedes dominam estatísticas ofensivas do Corinthians no Paulistão'),
 ('h3',
  'Santos tem apenas 11%

### Buscando links

Um **tag** específico muito útil na construção de **web crawlers** é o `<a>`. Este **tag** contém os hiper-links de uma página HTML. Vamos utilizar a BeautifulSoup para extrair todas os links de nossa notícia.

In [36]:
tag_a = soup.find_all("a")


In [37]:
tag_a[0:5]


[<a data-audience-click='{"component":"barra-uol","reference":"item-ingresso.com","position":"coluna-unica","area":"barra-uol","mediaName":"Home"}' href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom">Ingresso.com</a>,
 <a data-audience-click='{"component":"barra-uol","reference":"item-batepapo","position":"coluna-unica","area":"barra-uol","mediaName":"Home"}' href="https://batepapo.uol.com.br/?utm_source=midia-interna_uol.com.br&amp;utm_medium=barrauol-internas&amp;utm_campaign=linkfixo_barrauol&amp;utm_term=barrauol-uolplay&amp;utm_content=barrauol">BATE-PAPO</a>,
 <a data-audience-click='{"component":"barra-uol","reference":"item-uol-meunegocio","position":"coluna-unica","area":"barra-uol","mediaName":"Home"}' href="https://meunegocio.uol.com.br/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo-barrauol-umn&amp;utm_term=barra

In [38]:
first_link = tag_a[0]


In [39]:
first_link


<a data-audience-click='{"component":"barra-uol","reference":"item-ingresso.com","position":"coluna-unica","area":"barra-uol","mediaName":"Home"}' href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom">Ingresso.com</a>

O atributo `.text` não extrai o URL do link, apenas o texto que é exibido para o usuário:

In [40]:
first_link.text


'Ingresso.com'

Se olharmos o *string* do **tag** poderemos entender melhor porque isso acontece. O URL em si está **dentro da declaração do tag**:

    <a data-audience-click="" href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom">

Dentro da declaração do **tag** podemos ver algo parecido com a declaração de variáveis:

    data-audience-click=""

e

    href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom"

Cada uma dessas *variáveis* é um **atributo do tag** e para extrai-las utilizaremos o atributo `.attrs`:

In [41]:
first_link.attrs


{'href': 'https://www.ingresso.com/?utm_source=uol.com.br&utm_medium=barrauol&utm_campaign=linkfixo_barrauol&utm_content=barrauol-link-ingressocom&utm_term=barrauol-ingressocom',
 'data-audience-click': '{"component":"barra-uol","reference":"item-ingresso.com","position":"coluna-unica","area":"barra-uol","mediaName":"Home"}'}

Podemos ver que os atributos de um **tag** são retornados como um **dict**! Se quisermos acessar uma **variável** específica podemos faze-lo através do nome desta variável:

In [42]:
first_link.attrs["href"]


'https://www.ingresso.com/?utm_source=uol.com.br&utm_medium=barrauol&utm_campaign=linkfixo_barrauol&utm_content=barrauol-link-ingressocom&utm_term=barrauol-ingressocom'

In [44]:
len(tag_a)

438

In [46]:
# EXERCICIO
# Utilize um loop para extrair todos os URLs de
# tags <a>
# lista_links = [link.attrs["href"] for link in tag_a]

lista_links = []
for link in tag_a:
    try:
        lista_links.append(link.attrs["href"])
    except KeyError as e:
        print("Este tag não contém um link!")


Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!
Este tag não contém um link!


In [47]:
lista_links

['https://www.ingresso.com/?utm_source=uol.com.br&utm_medium=barrauol&utm_campaign=linkfixo_barrauol&utm_content=barrauol-link-ingressocom&utm_term=barrauol-ingressocom',
 'https://batepapo.uol.com.br/?utm_source=midia-interna_uol.com.br&utm_medium=barrauol-internas&utm_campaign=linkfixo_barrauol&utm_term=barrauol-uolplay&utm_content=barrauol',
 'https://meunegocio.uol.com.br/?utm_source=uol.com.br&utm_medium=barrauol&utm_campaign=linkfixo-barrauol-umn&utm_term=barrauol-umn&utm_content=barrauol-umn',
 'https://www.passeidireto.com/?utm_source=uol.com.br&utm_medium=barra-uol-interna',
 'https://pagseguro.uol.com.br/?utm_source=midia-interna&utm_medium=barra_uol&utm_campaign=link_fixo_ps&utm_content=estacao',
 'https://play.uol.com.br/?utm_source=uol.com.br&utm_medium=barrauol&utm_campaign=linkfixo_barrauol&utm_term=barrauol-uolplay&utm_content=barrauol',
 'https://www.uol.com.br/',
 'https://www.uol.com.br/esporte/',
 'https://sac.uol.com.br/',
 'https://mail.uol.com.br/checkin',
 ' htt

## **Tags** hierárquicos

Até agora todos os **tags** que vimos eram **simples**, ou seja, não continham em seu conteúdo outros **tags**. Muitas vezes queremos extrair informações *navegando* a página **bloco a bloco**.

Vamos aprender a utilizar o **inspetor de código** de um web browser para navegar blocos de tags para extrair informações complexas. Neste exemplo vamos reconstruir um tabela da Wikipedia com um DataFrame.

In [50]:
url = "https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy"


In [51]:
response = requests.get(url)
html = response.content


In [52]:
soup = BeautifulSoup(html)


In [53]:
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of European countries by life expectancy - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feat

Vamos acessar o link [List of European countries by life_expectancy](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy) para entender como podemos utilizar o inspetor de código fonte para descobrir onde em nosso HTML está nossa tabela!

### Extraindo a tabela completa

Com o inspetor podemos ver que as tabelas desta página estão todas dentro de tags `table`. 

In [54]:
table_wiki_raw = soup.find_all("table")


In [55]:
len(table_wiki_raw)


14

Infelizmente, esta **tag** é muitas vezes utilizada como elemento de formatação de páginas (especialmente em página antigas).

Podemos utilizar o **argumento** `attrs =` do método `.find_all()` para **filtrar** os **tags** extraídos a partir dos **valores de seus atributos**.

O atributo `class` é um ótimo candidato em varias ocasiões para esse tipo de filtro. No caso da nossa tabela atual podemos ver que a classe é:

    wikitable sortable static-row-numbers plainrowheaders srn-white-background jquery-tablesorter

Vamos ver como utilizar o `attrs =` para realizar esse filtro.

In [56]:
table_wiki_raw = soup.find_all("table", attrs={"class": "wikitable"})


In [57]:
len(table_wiki_raw)


4

In [58]:
print(table_wiki_raw[0])


<table border="1" class="wikitable sortable static-row-numbers plainrowheaders srn-white-background" style="text-align:right;">
<tbody><tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th>Countries
</th>
<th style="width:4em;">all
</th>
<th style="width:4em;">male
</th>
<th style="width:4em;">female
</th>
<th style="width:4em;"><abbr title="Difference in life expectancy between females and males">gender<br/>gap</abbr>
</th>
<th style="width:4em; border-left-width:2px;"><abbr title="Improvement of the indicator for all population compared to the previous year">Δ 2019<br/>all</abbr>
</th>
<th style="width:4em;"><abbr title="Improvement of the indicator for male compared to the previous year">Δ 2019<br/>male</abbr>
</th>
<th style="width:4em;"><abbr title="Improvement of the indicator for female compared to the previous year">Δ 2019<br/>female</abbr>
</th>
<th style="width:5em;"><abbr title="Increase of gender gap compared to the previous year">Δ 2019<br/>ge

In [59]:
type(table_wiki_raw[0])

bs4.element.Tag

### Navegando um **tag hierárquico**

Além de poder percorrer o HTML completo utilizando o método `.find_all()`, podemos fazer buscas dentro de cada **tag**! Isso nos permite **localizar** a busca em um bloco específico delimitado por **tags**.

No nosso caso atual, podemos ver que cada tabela extraída é composta por **três tags**: `<thead>`, `<tbody>` e `<tfoot>`. Como queremos extrair o **corpo** da tabela, vamos investigar o tag `tbody`

In [60]:
table_wiki = table_wiki_raw[0].find("tbody")


In [71]:
print(table_wiki_raw[0].prettify())

<table border="1" class="wikitable sortable static-row-numbers plainrowheaders srn-white-background" style="text-align:right;">
 <tbody>
  <tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
   <th>
    Countries
   </th>
   <th style="width:4em;">
    all
   </th>
   <th style="width:4em;">
    male
   </th>
   <th style="width:4em;">
    female
   </th>
   <th style="width:4em;">
    <abbr title="Difference in life expectancy between females and males">
     gender
     <br/>
     gap
    </abbr>
   </th>
   <th style="width:4em; border-left-width:2px;">
    <abbr title="Improvement of the indicator for all population compared to the previous year">
     Δ 2019
     <br/>
     all
    </abbr>
   </th>
   <th style="width:4em;">
    <abbr title="Improvement of the indicator for male compared to the previous year">
     Δ 2019
     <br/>
     male
    </abbr>
   </th>
   <th style="width:4em;">
    <abbr title="Improvement of the indicator for female compare

In [61]:
print(table_wiki.prettify())


<tbody>
 <tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
  <th>
   Countries
  </th>
  <th style="width:4em;">
   all
  </th>
  <th style="width:4em;">
   male
  </th>
  <th style="width:4em;">
   female
  </th>
  <th style="width:4em;">
   <abbr title="Difference in life expectancy between females and males">
    gender
    <br/>
    gap
   </abbr>
  </th>
  <th style="width:4em; border-left-width:2px;">
   <abbr title="Improvement of the indicator for all population compared to the previous year">
    Δ 2019
    <br/>
    all
   </abbr>
  </th>
  <th style="width:4em;">
   <abbr title="Improvement of the indicator for male compared to the previous year">
    Δ 2019
    <br/>
    male
   </abbr>
  </th>
  <th style="width:4em;">
   <abbr title="Improvement of the indicator for female compared to the previous year">
    Δ 2019
    <br/>
    female
   </abbr>
  </th>
  <th style="width:5em;">
   <abbr title="Increase of gender gap compared to the previous

Podemos ver tanto no inspetor quanto no print acima que o corpo da tabela é composto por múltiplos tags `<tr>`: se olharmos com cuidado veremos que cada `<tr>` é uma linha de nossa tabela! Vamos extrair uma linha em particular para ver do que ela é composta.

In [63]:
table_rows = table_wiki.find_all("tr")


In [69]:
table_rows

[<tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
 <th>Countries
 </th>
 <th style="width:4em;">all
 </th>
 <th style="width:4em;">male
 </th>
 <th style="width:4em;">female
 </th>
 <th style="width:4em;"><abbr title="Difference in life expectancy between females and males">gender<br/>gap</abbr>
 </th>
 <th style="width:4em; border-left-width:2px;"><abbr title="Improvement of the indicator for all population compared to the previous year">Δ 2019<br/>all</abbr>
 </th>
 <th style="width:4em;"><abbr title="Improvement of the indicator for male compared to the previous year">Δ 2019<br/>male</abbr>
 </th>
 <th style="width:4em;"><abbr title="Improvement of the indicator for female compared to the previous year">Δ 2019<br/>female</abbr>
 </th>
 <th style="width:5em;"><abbr title="Increase of gender gap compared to the previous year">Δ 2019<br/>gen. gap</abbr>
 </th></tr>,
 <tr class="static-row-header">
 <th style="background-position:center;line-height:70%;pad

In [ ]:
print(table_rows[1].prettify())


Cada linha de nossa tabela é composta por uma série de tags `<td>` - e cada um destes contém as informações de uma célula. Finalmente chegamos no elemento que contém os dados da tabela!

In [74]:
table_rows[2]

<tr>
<td align="left"><span class="flagicon" style="display:inline-block;width:25px;"><img alt="" class="thumbborder" data-file-height="372" data-file-width="512" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/21px-Flag_of_Norway.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/32px-Flag_of_Norway.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/41px-Flag_of_Norway.svg.png 2x" width="21"/></span> <a href="/wiki/Norway" title="Norway">Norway</a></td>
<td style="text-align:center; background:#e0ffd8;">83.21</td>
<td style="text-align:center; background:#eaf3ff;">81.60</td>
<td style="text-align:center; background:#fee7f6;">84.90</td>
<td style="text-align:center;">3.30</td>
<td style="text-align:center; background:#e0ffd8; border-left-width:2px;">0.25</td>
<td style="text-align:center; background:#eaf3ff;">0.30</td>
<td style="text-align:center; background:

In [75]:
first_row = table_rows[2].find_all("td")


In [77]:
first_row[0]


<td align="left"><span class="flagicon" style="display:inline-block;width:25px;"><img alt="" class="thumbborder" data-file-height="372" data-file-width="512" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/21px-Flag_of_Norway.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/32px-Flag_of_Norway.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/41px-Flag_of_Norway.svg.png 2x" width="21"/></span> <a href="/wiki/Norway" title="Norway">Norway</a></td>

In [78]:
[elemento.text.strip() for elemento in first_row]


['Norway', '83.21', '81.60', '84.90', '3.30', '0.25', '0.30', '0.20', '−0.10']

Vamos juntar todas as etapas acima em um loop (ou list comprehension) para construir nosso DataFrame!

In [79]:
table_wiki

<tbody><tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th>Countries
</th>
<th style="width:4em;">all
</th>
<th style="width:4em;">male
</th>
<th style="width:4em;">female
</th>
<th style="width:4em;"><abbr title="Difference in life expectancy between females and males">gender<br/>gap</abbr>
</th>
<th style="width:4em; border-left-width:2px;"><abbr title="Improvement of the indicator for all population compared to the previous year">Δ 2019<br/>all</abbr>
</th>
<th style="width:4em;"><abbr title="Improvement of the indicator for male compared to the previous year">Δ 2019<br/>male</abbr>
</th>
<th style="width:4em;"><abbr title="Improvement of the indicator for female compared to the previous year">Δ 2019<br/>female</abbr>
</th>
<th style="width:5em;"><abbr title="Increase of gender gap compared to the previous year">Δ 2019<br/>gen. gap</abbr>
</th></tr>
<tr class="static-row-header">
<th style="background-position:center;line-height:70%;padding-top:0"> </

In [83]:
# EXERCICIO
# Construir um DataFrame com os dados da tabela de Expectativa de Vida na Europa da Wikipedia
# BONUS - faça isso com um list comprehension!
# BONUS - contrua uma função que receba um link da Wikipedia com uma tabela e retorne um DataFrame (teste em outra tabela)
lista_tabela = []
for row in table_wiki.find_all("tr"):
    lista_row = []
    data_elem = row.find_all("td")
    if data_elem:
        for celula in data_elem:
            lista_row.append(celula.text)
        lista_tabela.append(lista_row)

In [85]:
pd.DataFrame(lista_tabela)

,0,1,2,3,4,5,6,7,8
0,Norway,83.21,81.60,84.90,3.30,0.25,0.30,0.20,−0.10\n
1,Switzerland,83.10,81.10,85.20,4.10,−0.80,−1.00,−0.60,0.40\n
2,Faroe Islands,83.09,80.80,85.50,4.70,0.30,0.30,0.30,0.00\n
3,Iceland,83.07,81.70,84.50,2.80,−0.10,0.00,−0.20,−0.20\n
4,Malta,82.65,80.80,84.60,3.80,−0.20,−0.40,0.00,0.40\n
5,Sweden,82.41,80.70,84.20,3.50,−0.70,−0.80,−0.60,0.20\n
6,Italy,82.34,80.10,84.70,4.60,−1.15,−1.30,−1.00,0.30\n
7,Spain,82.33,79.70,85.10,5.40,−1.50,−1.40,−1.60,−0.20\n
8,Ireland,82.20,80.40,84.10,3.70,−0.50,−0.40,−0.60,−0.20\n
9,Gibraltar,82.20,80.25,84.11,3.86,−0.17,−0.12,−0.18,−0.05\n


# Criando um WebCrawler

Um **webcrawler** é uma aplicação que extrai informações estruturadas a partir de multiplos sites de forma autonôma. Vamos construir um **webcrawler** para extrair um artigo (artigo origem) do jornal Guardian e qualquer outro artigo referenciado no texto do artigo origem (artigos filhos).

## Extraindo o Artigo Origem

O primeiro passo do nosso *crawler* é encontrar o corpo principal do artigo - caso contrário *puxaremos* links de propagandas, artigos relacionados, cabeçalho, etc...

In [86]:
url = "https://www.theguardian.com/world/2022/aug/18/fires-and-explosions-reported-at-military-targets-in-russia-and-crimea"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)


Vamos começar buscando algum **tag** que represente o corpo do artigo:

In [87]:
artigo = soup.find("article")
print(artigo.text)


00:49Russia: huge fire at ammunition depot near border with Ukraine – videoRussia This article is more than 6 months oldFires and explosions reported at military targets in Russia and CrimeaThis article is more than 6 months oldMunitions depot in Belgorod province and airbase near Sevastopol hit in latest apparent sabotage missions

See all our Ukraine war coverage
Emma Graham-Harrison in KyivFri 19 Aug 2022 18.08 BSTFirst published on Thu 18 Aug 2022 23.55 BSTFires and explosions have been reported at military targets inside Russia and Russian-occupied parts of Ukraine, in the latest of a string of apparent sabotage missions deep inside Russian-held territory as western officials suggested the conflict had reached deadlock.Two Russian villages were evacuated after a blaze at a munitions depot near the Ukrainian border in Belgorod province. “An ammunition depot caught fire near the village of Timonovo”, less than 30 miles (50km) from the border, the regional governor, Vyacheslav Gladko

Agora que encontramos um **tag** com o corpo do artigo precisamos encontrar links dentro deste corpo:

In [88]:
links_artigo = artigo.find_all("a", attrs={"data-link-name": "in body link"})
len(links_artigo)


4

Podemos utilizar list comprehensions para visualizar para onde esses links nos levarão:

In [90]:
[link["href"] for link in links_artigo]


['https://www.theguardian.com/world/2022/aug/16/ukraine-hints-it-was-behind-latest-attack-on-russian-supply-lines-in-crimea',
 'https://www.theguardian.com/world/2022/aug/10/ukraine-air-force-claims-russian-jets-destroyed-crimea-raid',
 'https://www.theguardian.com/world/2022/aug/16/ukraine-hints-it-was-behind-latest-attack-on-russian-supply-lines-in-crimea',
 'https://www.theguardian.com/world/volodymyr-zelenskiy']

Sempre que precisamos construir um loop para executar um bloco de código sobre os elementos de uma lista devemos **testar** esse bloco de código em um elemento particular da lista antes de contruir o loop completo:

In [91]:
url_filho = links_artigo[0]["href"]


Da mesma forma que extraímos o artigo original, podemos extrair o artigo filho a partir do link extraído:

In [92]:
response_filho = requests.get(url_filho)
html_filho = response_filho.content
soup_filho = BeautifulSoup(html_filho)


In [93]:
artigo_filho = soup_filho.find("article").text
print(artigo_filho)


00:45Footage purports to show explosion at ammunition depot in Crimea – videoUkraine This article is more than 6 months oldUkraine hints it was behind latest attack on Russian supply lines in CrimeaThis article is more than 6 months oldWhile not formally confirming responsibility for mystery strike, Kyiv officials react with glee on social media

Russia-Ukraine war: latest news
Luke Harding in KyivTue 16 Aug 2022 15.56 BSTFirst published on Tue 16 Aug 2022 13.22 BSTUkraine has hinted it was behind a series of mysterious and devastating strikes in occupied Crimea that destroyed a key railway junction used for supplying Russian troops and a military airbase.Smoke billowed into the sky near Dzhankoi, a significant railway hub in the north of the peninsula used by the Russian military to transport troops and equipment to occupied Melitopol, which Moscow seized early in its full-scale invasion.Several explosions on Tuesday appeared to have destroyed a Russian ammunition depot and an electri

Agora podemos consolidar nosso código para executar o bloco acima sobre todos os *links* extraídos do artigo origem (adicionando algumas camadas de segurança para que nosso *crawler* navegue tranquilamente):

In [95]:
[link["href"] for link in links_artigo]

['https://www.theguardian.com/world/2022/aug/16/ukraine-hints-it-was-behind-latest-attack-on-russian-supply-lines-in-crimea',
 'https://www.theguardian.com/world/2022/aug/10/ukraine-air-force-claims-russian-jets-destroyed-crimea-raid',
 'https://www.theguardian.com/world/2022/aug/16/ukraine-hints-it-was-behind-latest-attack-on-russian-supply-lines-in-crimea',
 'https://www.theguardian.com/world/volodymyr-zelenskiy']

In [94]:
lista_artigos = []
lista_links = [link["href"] for link in links_artigo]
for link in lista_links:
    if link.find("guardian") >= 0:
        try:
            response_filho = requests.get(link)
            html_filho = response_filho.content
            soup_filho = BeautifulSoup(html_filho)
            lista_artigos.append(soup_filho.find("article").text)
        except AttributeError:
            continue


In [98]:
lista_artigos[2]


'00:45Footage purports to show explosion at ammunition depot in Crimea – videoUkraine This article is more than 6 months oldUkraine hints it was behind latest attack on Russian supply lines in CrimeaThis article is more than 6 months oldWhile not formally confirming responsibility for mystery strike, Kyiv officials react with glee on social media\n\nRussia-Ukraine war: latest news\nLuke Harding in KyivTue 16 Aug 2022 15.56 BSTFirst published on Tue 16 Aug 2022 13.22 BSTUkraine has hinted it was behind a series of mysterious and devastating strikes in occupied Crimea that destroyed a key railway junction used for supplying Russian troops and a military airbase.Smoke billowed into the sky near Dzhankoi, a significant railway hub in the north of the peninsula used by the Russian military to transport troops and equipment to occupied Melitopol, which Moscow seized early in its full-scale invasion.Several explosions on Tuesday appeared to have destroyed a Russian ammunition depot and an ele

In [ ]:
# EXERCICIO
# Transforme o código acima em uma função que recebe um link de artigo do Guardian como argumento
# e retorna uma lista com o texto do artigo original e o texto de quaisquer artigos do Guardian
# linkados no texto do artigo original
# BONUS - crie uma função que faça isso em profundidade maior que um: além do artigo original e dos artigos
# filhos (artigos linkados no artigo original) trazer os artigos netos (artigos linkados nos artigos filhos)
# BONUS BONUS - contrua uma função capaz de extrair uma profundidade arbitraria de artigos (filhos, netos, bisnetos...)
# a partir de um parametro 'depth' (0 = original, 1 = filhos, 2 = netos, 3 = bisnetos, etc)


# Conhecendo o Selenium

Embora o BeautifulSoup seja uma biblioteca ótima e simples para extrair dados de páginas (o complicado são os html's...), nem sempre conseguimos usa-la: muitas páginas utilizam, hoje em dia, tecnologias incompatíveis com a arquitetura do BeautifulSoup.

Páginas que são dinâmicas (especificamente, páginas que usem tecnologias client-side, como React.js) não podem ser mineradas diretamente.

Para tratar destas páginas precisamos utilizar outra biblioteca: Selenium. Enquanto na BeautifulSoup carregamos o HTML da página original para dentro do Python, com a Selenium **simularemos o ato de navegação**.

In [ ]:
!pip3 install selenium
!pip3 install webdriver-manager

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|█████████████████| 8.84M/8.84M [00:00<00:00, 11.5MB/s]
/var/folders/c2/4vv1dtt97gl9bn1lvw8rf0k40000gn/T/ipykernel_62045/80896275.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


Vamos utilizar nosso `driver` para navegar até uma página teste:

In [4]:
driver.get("https://testpages.herokuapp.com/styled/index.html")


## Navegando utilizando XPATHs

O forma mais fácil de navegar uma página utilizando Selenium é através de XPATHs: identificadores únicos dos elementos de uma página. Vamos ver como podemos descobrir um XPATH de um elemento particular e como utilizar o nosso `driver` para interagir com o elemento.

In [ ]:
"/html/body/div/ul[1]/li[4]/ul/li[1]/a"

In [5]:
link_htmlformtest = driver.find_element(
    By.XPATH, "/html/body/div/ul[1]/li[7]/ul/li[1]/a"
)


O método `.find_element()` nos permite encontrar elementos em uma página (parecido com o `.find_all()` do BS) - mas por si só não nos diz muito sobre o elemento:

In [6]:
link_htmlformtest


<selenium.webdriver.remote.webelement.WebElement (session="87a886506ae45e49094664b8fcf73902", element="cb000f8e-ce90-4d6c-9eb6-c27887614b3c")>

Podemos buscar os atributos do elemento utilizando o método `.get_attribute()` - vamos utilizar este método para extrair o URL do link associado ao elemento:

In [7]:
link_htmlformtest.get_attribute("href")


'https://testpages.herokuapp.com/styled/basic-html-form-test.html'

Até agora tudo está muito parecido com a BS - agora vamos interagir com a página, *clicando* virtualmente no link:

In [8]:
link_htmlformtest.click()


### Interagindo com Formulários

Um dos principais usos para o Selenium é conseguir preencher de forma programática formulários em páginas. Vamos continuar utilizando nosso driver para preencher o formulário na página para qual navegamos.

In [9]:
input_username = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[1]/td/input"
)


Podemos utilizar o método `.send_keys()` para *preencher* um formulário: 

In [10]:
input_username.send_keys("aaaa")


e o método `.clear()` para *limpar* o formulário:

In [11]:
input_username.clear()


Como a variável `input_username` ainda é o mesmo elemento (o campo **Username**), podemos voltar a preencher o formulário:

In [12]:
nome_usuario = "pedrotechel"
input_username.send_keys(nome_usuario)


Elementos clicáveis, como botões ou checkboxes podem ser acessados através do método click:

In [15]:
input_opt1 = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[5]/td/input[1]"
)
input_opt1.click()


In [16]:
input_rdbx1 = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[6]/td/input[1]"
)
input_rdbx1.click()


## Exemplo Prático - Mercado Livre

Vamos construir um *webcrawler*, baseado em Selenium, para buscar preços de azeite no Mercado Livre. Plataformas de e-commerce quase sempre utilizam tecnologias que impossibilitam a utilização do BeautifulSoup!

Além de *apontar* o driver para a página principal do Mercado Livre, vamos utilizar o método `.implicitly_wait()` para que o driver *espere* um tempo até todos os elementos da página renderizarem:

In [17]:
driver.get("https://www.mercadolivre.com.br/")
driver.implicitly_wait(10)


Agora vamos interagir com a barra de busca para procurar azeites. Muitas barras de busca tem testos pré-preenchidos, então antes de continuarmos com nossa busca vamos limpar a barra utilizando o método `.clear()`:

In [ ]:
/html/body/header/div/div[2]/form/input

In [18]:
barra_busca = driver.find_element(By.XPATH, "/html/body/header/div/div[2]/form/input")
barra_busca.clear()


Com a barra limpa, podemos utilizar o método `.send_keys()` para preenche-la com o nome do produto que queremos buscar (`"azeite"`) e confirmar a busca utilizando o objeto `Keys.ENTER` (simulando digitar "azeite" e apertar *enter* na barra de busca): 

In [19]:
barra_busca.send_keys("azeite")

In [20]:
barra_busca.send_keys(Keys.ENTER)


Até agora tranquilo! Para continuarmos, no entanto, precisaremos extrair TODOS os preços de produtos. Para isso utilizaremos o método `.find_elements()` (no plural) e, ao invés de utilizar o XPATH, buscaremos pelo **tag** e sua **classe**. Vamos começar encontrando todas as *caixas de produto*

In [21]:
lista_elem_produto = driver.find_elements(
    By.CLASS_NAME, "ui-search-result__content-wrapper"
)
len(lista_elem_produto)


54

In [22]:
lista_elem_produto[0]

<selenium.webdriver.remote.webelement.WebElement (session="87a886506ae45e49094664b8fcf73902", element="45b31a9a-6959-4f1e-9116-7e68740e847a")>

A lista construída acima contém `WebElements` - da mesma forma que podemos fazer buscas dentro de **tags** hierarquicas utilizando o BS, podemos fazer buscas dentro de diferentes `WebElements` utilizando o Selenium:

In [23]:
lista_elem_preco = []
for produto in lista_elem_produto:
    caixa_preco = produto.find_element(By.CLASS_NAME, "price-tag-amount")
    lista_elem_preco.append(caixa_preco)


In [26]:
len(lista_elem_preco)

54

In [28]:
lista_elem_preco[1].text

'R$\n25\n,\n99'

Vamos utilizar o atributo `.text` para extrair o conteúdo de cada uma das caixas de preço (primeiro testando com um elemento da lista):

In [29]:
preco_teste = lista_elem_preco[0]
preco_teste.text


'R$\n29\n,\n39'

O preço extraído acima parece não bater com o preço da página! No entanto, se prestarmos atenção, veremos que este é o preço sem desconto do produto (preço cheio).

Vamos alterar nosso loop para extrair todos os preços de cada produto (cheio e descontado):

In [30]:
lista_elem_preco = []
for produto in lista_elem_produto:
    caixa_preco = produto.find_elements(By.CLASS_NAME, "price-tag-amount")
    lista_elem_preco.append(caixa_preco)


In [31]:
lista_elem_preco[0]


[<selenium.webdriver.remote.webelement.WebElement (session="87a886506ae45e49094664b8fcf73902", element="9ca45a2e-0113-49fc-a52e-c42bb2b1d43b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="87a886506ae45e49094664b8fcf73902", element="35eecc19-a42a-4093-9777-3768d2471e1f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="87a886506ae45e49094664b8fcf73902", element="9ceb25a3-90e1-4453-8e4b-9f4f4dcc5078")>]

Temos 3 preços associados à este produto... Vamos analisar o que cada um deles representa:

In [32]:
[preco.text for preco in lista_elem_preco[0]]


['R$\n29\n,\n39', 'R$\n23\n,\n89', 'R$\n2\n,\n32']

O primeiro preço representa o preço cheio, o segundo com desconto e o terceiro é o preço de cada parcela. Este último não representa muita coisa, já que o número de parcelas pode variar entre produtos. Vamos tratar os strings dos dois primeiros, transformado-os em floats, em uma lista de preços:

In [33]:
import re


In [34]:
def limpar_preco(str_preco):
    pattern = r"[^0-9.,]"
    return float(re.sub(pattern, "", str_preco).replace(",", "."))


In [35]:
[limpar_preco(preco.text) for preco in lista_elem_preco[0]]


[29.39, 23.89, 2.32]

Agora vamos construir nosso loop!

In [36]:
lista_preco_cheio = []
lista_preco_desconto = []

for elem_preco in lista_elem_preco:
    precos = [limpar_preco(preco.text) for preco in elem_preco]
    lista_preco_cheio.append(precos[0])
    lista_preco_desconto.append(precos[1])

print(lista_preco_cheio)


[29.39, 25.99, 32.9, 29.39, 30.45, 29.9, 25.99, 46.75, 11.5, 29.9, 28.35, 35.89, 41.25, 29.39, 32.9, 9.49, 22.9, 17.9, 24.9, 36.63, 54.9, 213.59, 40.79, 73.87, 9.49, 30.45, 22.9, 10.9, 53.1, 73.87, 59.9, 36.75, 69.33, 197.99, 29.39, 39.89, 32.9, 11.9, 125.9, 13.95, 52.9, 29.9, 79.9, 79.9, 62.9, 31.44, 60.9, 92.9, 49.31, 16.99, 29.9, 69.9, 36.0, 279.99]


Vamos investigar a lista de preços com desconto:

In [37]:
print(lista_preco_desconto)

[23.89, 19.9, 22.79, 23.89, 24.99, 21.9, 19.9, 32.69, 1.11, 21.9, 24.29, 3.48, 28.89, 21.27, 24.47, 7.99, 15.79, 15.9, 17.9, 3.55, 47.9, 20.76, 32.9, 7.18, 7.99, 23.19, 15.79, 8.29, 17.7, 7.18, 5.82, 25.69, 50.31, 19.24, 23.89, 27.89, 25.9, 8.49, 12.24, 1.35, 44.96, 21.9, 59.9, 59.9, 6.11, 21.38, 5.92, 9.03, 44.37, 1.65, 2.9, 6.79, 3.49, 27.15]


In [39]:
[len(preco)for preco in lista_elem_preco]

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2]

Alguns preços estão muito abaixo do esperado! Se compararmos os resultados com a página veremos que nem todos os produtos tem descontos. Vamos investigar um preço que tenha essa caracteristica:

In [40]:
[limpar_preco(preco.text) for preco in lista_elem_preco[1]]

[25.99, 19.9, 1.93]

Quando um produto não está promocionado, o segundo elemento da lista preço é o valor da parcela para compras parceladas! Vamos alterar nosso loop para contemplar essa condição, guardando `np.nan` no preço descontado de produtos sem desconto:

In [41]:
lista_preco_cheio = []
lista_preco_desconto = []

for elem_preco in lista_elem_preco:
    if len(elem_preco) == 3:
        precos = [limpar_preco(preco.text) for preco in elem_preco]
        lista_preco_cheio.append(precos[0])
        lista_preco_desconto.append(precos[1])
    else:
        print("Produto sem promoção!")
        precos = [limpar_preco(preco.text) for preco in elem_preco]
        lista_preco_cheio.append(precos[0])
        lista_preco_desconto.append(np.nan)


Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!
Produto sem promoção!


In [42]:
lista_preco_desconto

[23.89,
 19.9,
 22.79,
 23.89,
 24.99,
 21.9,
 19.9,
 32.69,
 nan,
 21.9,
 24.29,
 nan,
 28.89,
 21.27,
 24.47,
 7.99,
 15.79,
 15.9,
 17.9,
 nan,
 47.9,
 nan,
 32.9,
 nan,
 7.99,
 23.19,
 15.79,
 8.29,
 nan,
 nan,
 nan,
 25.69,
 50.31,
 nan,
 23.89,
 27.89,
 25.9,
 8.49,
 nan,
 nan,
 44.96,
 21.9,
 59.9,
 59.9,
 nan,
 21.38,
 nan,
 nan,
 44.37,
 nan,
 nan,
 nan,
 nan,
 nan]

Podemos utilizar o mesmo método para construir a lista de nomes de produto:

In [43]:
lista_elem_produto[0]

<selenium.webdriver.remote.webelement.WebElement (session="87a886506ae45e49094664b8fcf73902", element="45b31a9a-6959-4f1e-9116-7e68740e847a")>

In [44]:
( 
    lista_elem_produto[0]
    .find_element(By.CLASS_NAME, "ui-search-item__title")
    .text
)

'Azeite de Oliva Extra Virgem Português Andorinha Clássicos Vidro 500ml'

In [45]:
lista_nome = []
for elem_nome in lista_elem_produto:
    nome = elem_nome.find_element(By.CLASS_NAME, "ui-search-item__title")
    lista_nome.append(nome.text)


Agora vamos juntar nossas duas listas em um DataFrame:

In [46]:
tb_azeite = pd.DataFrame(
    {
        "nome": lista_nome,
        "preco_cheio": lista_preco_cheio,
        "preco_desconto": lista_preco_desconto,
    }
)
tb_azeite.head()


,nome,preco_cheio,preco_desconto
0,Azeite de Oliva Extra Virgem Português Andorin...,29.39,23.89
1,Óleo extra virgem de oliva Gomes da Costa vidr...,25.99,19.90
2,Azeite Chileno Extra Virgem O-live 500ml,32.90,22.79
3,Azeite de Oliva Extra Virgem Ouro 1927 Portugu...,29.39,23.89
4,Azeite Italiano Extra Virgem Filippo Berio 500ml,30.45,24.99


In [50]:
np.quantile(tb_azeite['preco_cheio'], [0.1, 0.9])

array([14.862, 79.9  ])